# Combining & organizing data

## Libraries and settings

In [1]:
# Libraries
import os
import pandas as pd
import numpy as np
import fnmatch

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_03


## Importing the prepared rental apartments data

In [2]:
# Get current working directory
print(os.getcwd())

# Read data to pandas data frame
df_orig = pd.read_csv('apartments_data_prepared.csv', 
                      sep=',', 
                      encoding='utf-8')

# Copy of data with selected colums
columns = ['web-scraper-order', 
              'address_raw', 
              'datetime', 'rooms', 
              'area', 
              'luxurious',
              'price',
              'price_per_m2']
df = df_orig[columns]

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

/workspaces/data_analytics/Week_03
(914, 8)


,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2023-08-21 09:03:47,3.5,122.0,1,3180.0,26.07
1,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2023-08-21 09:03:47,2.5,78.0,0,3760.0,48.21
2,1662023742-807,"Langfurrenstrasse 5c, 8623 Wetzikon ZH, ZH",2023-08-21 09:03:47,5.5,115.0,0,2860.0,24.87
3,1662023804-1290,"Sandbuckweg 5A, 8157 Dielsdorf, ZH",2023-08-21 09:03:47,3.5,74.0,0,2165.0,29.26
4,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2023-08-21 09:03:47,5.5,195.0,0,6900.0,35.38


## Combining data from different sources

### Reading rental apartment data with geocoded addresses

In [3]:
# Meaning of variables
# lat: geographical latitude
# lon: geographical longitude
# bfs_number: official municipality id
# bfs_name: official municipality name

# Geocoded data (i.e. data with latitude and longitude)
df_geo = pd.read_csv('apartments_data_geocoded.csv', 
                     sep=',', 
                     encoding='utf-8')
df_geo.head(5)

,web-scraper-order,address_raw,lat,lon,bfs_number,bfs_name
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",47.255714,8.804976,112,Bubikon
1,1662023720-634,"Blumenbergstrasse 7, 8633 Wolfhausen, ZH",47.254879,8.793746,112,Bubikon
2,1662023745-834,"8608 Bubikon, ZH",47.277386,8.800306,112,Bubikon
3,1662023701-503,"8608 Bubikon, ZH",47.277386,8.800306,112,Bubikon
4,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",47.361378,8.533339,261,Zürich


### Join geo-information to rental apartment data using .merge()

In [4]:
df2 = df.merge(df_geo[['web-scraper-order', 
                       'lat', 
                       'lon', 
                       'bfs_number', 
                       'bfs_name']], 
               on="web-scraper-order")
df2.head()

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2023-08-21 09:03:47,3.5,122.0,1,3180.0,26.07,47.255714,8.804976,112,Bubikon
1,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2023-08-21 09:03:47,2.5,78.0,0,3760.0,48.21,47.361378,8.533339,261,Zürich
2,1662023742-807,"Langfurrenstrasse 5c, 8623 Wetzikon ZH, ZH",2023-08-21 09:03:47,5.5,115.0,0,2860.0,24.87,47.328632,8.810400,121,Wetzikon (ZH)
3,1662023804-1290,"Sandbuckweg 5A, 8157 Dielsdorf, ZH",2023-08-21 09:03:47,3.5,74.0,0,2165.0,29.26,47.477493,8.456285,86,Dielsdorf
4,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2023-08-21 09:03:47,5.5,195.0,0,6900.0,35.38,47.366898,8.528817,261,Zürich


### Reading municipality data

In [5]:
# Meaning of variables:
# bfs_number: official municipality id
# bfs_name: official municipality name
# pop: number of residents (=population)
# pop_dens: population density (pop per km2)
# frg_pct: percentage foreigners
# emp: numer of employees

df_municip = pd.read_excel('municipality_data.xlsx', 
                           sheet_name='data_for_import')
df_municip.head(5)

,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,tax_income
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,105013.570634
1,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,71059.805603
2,3,Bonstetten,5572,749.932705,16.564968,1014.0,88927.698145
3,4,Hausen am Albis,3751,275.808824,16.022394,1021.0,86300.455137
4,5,Hedingen,3778,578.560490,16.410799,1478.0,90811.205330


### Join municipality data to rental apartment data using .merge()

In [6]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df3 = df2.merge(df_municip[['bfs_number', 
                            'pop', 
                            'pop_dens', 
                            'frg_pct', 
                            'emp',
                            'tax_income']], 
                on="bfs_number")
df3.head(5)

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,tax_income
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2023-08-21 09:03:47,3.5,122.0,1,3180.0,26.07,47.255714,8.804976,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340
1,1662023720-634,"Blumenbergstrasse 7, 8633 Wolfhausen, ZH",2023-08-21 09:03:47,4.0,87.0,0,1690.0,19.43,47.254879,8.793746,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340
2,1662023745-834,"8608 Bubikon, ZH",2023-08-21 09:03:47,3.5,92.0,0,2350.0,25.54,47.277386,8.800306,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340
3,1662023701-503,"8608 Bubikon, ZH",2023-08-21 09:03:47,3.5,130.0,0,2500.0,19.23,47.277386,8.800306,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340
4,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2023-08-21 09:03:47,2.5,78.0,0,3760.0,48.21,47.361378,8.533339,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704


### Export data to file

In [7]:
### Export apartment data to file
df3.to_csv('apartments_data_enriched.csv',
            sep=',',
            encoding='utf-8')

### Sorting data

In [8]:
# Sorting data by 'price' and 'area' with highest price above (ascending=False)
df3.sort_values(by=['price', 'area'], 
                ascending=False).head()

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,tax_income
76,1662023771-1030,"Parkring 57, 8002 Zürich, ZH",2023-08-21 09:03:47,5.0,187.0,0,8900.0,47.59,47.366734,8.528435,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
172,1662023689-385,"8053 Zürich, ZH",2023-08-21 09:03:47,4.5,171.0,0,7900.0,46.20,47.358616,8.578999,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
36,1662023733-742,"Giessereistrasse 12, 8005 Zürich, ZH",2023-08-21 09:03:47,2.5,282.0,0,7500.0,26.60,47.390221,8.518072,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
5,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2023-08-21 09:03:47,5.5,195.0,0,6900.0,35.38,47.366898,8.528817,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
18,1662023786-1149,"Freudenbergstrasse 94, 8044 Zürich, ZH",2023-08-21 09:03:47,4.5,128.0,0,6630.0,51.80,47.384132,8.556532,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704


### Aggregation of data

In [9]:
# Aggregation using .groupby()
df3[['rooms', 'price']].groupby(['rooms']).mean()

,price
rooms,
1.0,1501.477273
1.5,1904.409091
2.0,1562.805556
2.5,2273.563758
3.0,1825.421875
3.5,2339.687023
4.0,2261.100000
4.5,2650.764368
5.0,3257.230769


## Reshaping data

### Reshaping data using .stack() and .unstack()

In [10]:
df_sub = df3[['bfs_name', 'rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

df_sub_stacked = df_sub.stack()
print('Stacked')
print(df_sub_stacked, '\n')

# Using unstack
print('Unstacked (= back to original shape)')
print(df_sub_stacked.unstack())

Original shape
  bfs_name  rooms   price   area
0  Bubikon    3.5  3180.0  122.0
1  Bubikon    4.0  1690.0   87.0
2  Bubikon    3.5  2350.0   92.0
3  Bubikon    3.5  2500.0  130.0
4   Zürich    2.5  3760.0   78.0 

Stacked
0  bfs_name    Bubikon
   rooms           3.5
   price        3180.0
   area          122.0
1  bfs_name    Bubikon
   rooms           4.0
   price        1690.0
   area           87.0
2  bfs_name    Bubikon
   rooms           3.5
   price        2350.0
   area           92.0
3  bfs_name    Bubikon
   rooms           3.5
   price        2500.0
   area          130.0
4  bfs_name     Zürich
   rooms           2.5
   price        3760.0
   area           78.0
dtype: object 

Unstacked (= back to original shape)
  bfs_name rooms   price   area
0  Bubikon   3.5  3180.0  122.0
1  Bubikon   4.0  1690.0   87.0
2  Bubikon   3.5  2350.0   92.0
3  Bubikon   3.5  2500.0  130.0
4   Zürich   2.5  3760.0   78.0


### Reshaping data using .melt()

In [11]:
df_sub = df3[['rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

print('Reshaped using .melt()')
print(pd.melt(df, id_vars=['rooms'], value_vars=['price', 'area']))

Original shape
   rooms   price   area
0    3.5  3180.0  122.0
1    4.0  1690.0   87.0
2    3.5  2350.0   92.0
3    3.5  2500.0  130.0
4    2.5  3760.0   78.0 

Reshaped using .melt()
      rooms variable   value
0       3.5    price  3180.0
1       2.5    price  3760.0
2       5.5    price  2860.0
3       3.5    price  2165.0
4       5.5    price  6900.0
...     ...      ...     ...
1823    1.5     area    65.0
1824    3.5     area    70.0
1825    2.5     area    56.0
1826    3.5     area    70.0
1827    3.5     area    96.0

[1828 rows x 3 columns]


### Pivoting data using .pivot_table()

In [12]:
# Using pivot_table to reshape the data and calculate means 
pd.pivot_table(df3[['rooms', 'price', 'area', 'price_per_m2']],
               index=['rooms'],
               values=['price', 'area', 'price_per_m2'],
               aggfunc=(np.mean))

,area,price,price_per_m2
rooms,,,
1.0,43.113636,1501.477273,42.639545
1.5,61.909091,1904.409091,42.319091
2.0,55.194444,1562.805556,29.221667
2.5,68.704698,2273.563758,33.974430
3.0,68.781250,1825.421875,27.065625
3.5,87.240458,2339.687023,27.057863
4.0,87.360000,2261.100000,26.044000
4.5,110.764368,2650.764368,24.147644
5.0,124.769231,3257.230769,24.829231


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [14]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 5.15.0-1041-azure
Datetime: 2023-08-21 09:05:22
Python Version: 3.10.8
-----------------------------------
